# SciBERT for Multi-Label Classification

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/center-for-threat-informed-defense/tram/blob/main/user_notebooks/fine_tune_multi_label.ipynb)

This notebook allows one to continue fine-tuning our provided SciBERT-for-multilabel-sequence-classification on custom data.

To start, first select `Runtime > Change runtime type`, and under `Hardware accelerator` select `GPU`. Then run the next two cells. The first cell will download the model and the Python dependencies. The second cell will load the model and set up the selectors.

In [1]:
!mkdir scibert_multi_label_model
!wget https://ctidtram.blob.core.windows.net/tram-models/multi-label-20230803/config.json -O scibert_multi_label_model/config.json
!wget https://ctidtram.blob.core.windows.net/tram-models/multi-label-20230803/pytorch_model.bin -O scibert_multi_label_model/pytorch_model.bin
!pip install torch transformers pandas

--2025-09-06 13:22:56--  https://ctidtram.blob.core.windows.net/tram-models/multi-label-20230803/config.json
Resolving ctidtram.blob.core.windows.net (ctidtram.blob.core.windows.net)... 57.150.154.65
Connecting to ctidtram.blob.core.windows.net (ctidtram.blob.core.windows.net)|57.150.154.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2806 (2.7K) [application/json]
Saving to: ‘scibert_multi_label_model/config.json’

scibert_multi_label 100%[===================>]   2.74K  --.-KB/s    in 0s      

2025-09-06 13:22:57 (1.01 GB/s) - ‘scibert_multi_label_model/config.json’ saved [2806/2806]

--2025-09-06 13:22:57--  https://ctidtram.blob.core.windows.net/tram-models/multi-label-20230803/pytorch_model.bin
Resolving ctidtram.blob.core.windows.net (ctidtram.blob.core.windows.net)... 57.150.154.65
Connecting to ctidtram.blob.core.windows.net (ctidtram.blob.core.windows.net)|57.150.154.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43990

This cell instantiates the label encoder. Do not modify this cell, as the classes (ie, ATT&CK techniques) and their order must match those the model expects.

In [2]:
from sklearn.preprocessing import MultiLabelBinarizer as MLB

CLASSES = [
   'T1003.001', 'T1005', 'T1012', 'T1016', 'T1021.001', 'T1027',
   'T1033', 'T1036.005', 'T1041', 'T1047', 'T1053.005', 'T1055',
   'T1056.001', 'T1057', 'T1059.003', 'T1068', 'T1070.004',
   'T1071.001', 'T1072', 'T1074.001', 'T1078', 'T1082', 'T1083',
   'T1090', 'T1095', 'T1105', 'T1106', 'T1110', 'T1112', 'T1113',
   'T1140', 'T1190', 'T1204.002', 'T1210', 'T1218.011', 'T1219',
   'T1484.001', 'T1518.001', 'T1543.003', 'T1547.001', 'T1548.002',
   'T1552.001', 'T1557.001', 'T1562.001', 'T1564.001', 'T1566.001',
   'T1569.002', 'T1570', 'T1573.001', 'T1574.002'
]

mlb = MLB(classes=CLASSES)
mlb.fit([[c] for c in CLASSES])

mlb

MultiLabelBinarizer(classes=['T1003.001', 'T1005', 'T1012', 'T1016',
                             'T1021.001', 'T1027', 'T1033', 'T1036.005',
                             'T1041', 'T1047', 'T1053.005', 'T1055',
                             'T1056.001', 'T1057', 'T1059.003', 'T1068',
                             'T1070.004', 'T1071.001', 'T1072', 'T1074.001',
                             'T1078', 'T1082', 'T1083', 'T1090', 'T1095',
                             'T1105', 'T1106', 'T1110', 'T1112', 'T1113', ...])

This cell is for loading the training data. You will need to modify this cell to load your data. Ensure that by the end of this cell, a DataFrame has been assigned to the variable `data` that has a `sentence` column containing the sentences, and a `labels` column containing lists (or other container types) of strings, where those strings are the ATT&CK IDs that this model can classify. The lists can be empty for negative examples. It does not matter how the DataFrame is indexed or what other columns with other names, if any, it has.

For demonstration purposes, we will use the same multi-label data that was produced during this TRAM effort, even though the model was trained on this data already. This cell is only present to show the expected format of the `data` DataFrame, and is not intended to be run as shown.

In [4]:
import pandas as pd
data = pd.read_json('multi_label.json').drop(columns='doc_title').head(500)
data

,sentence,labels
0,title: NotPetya Technical Analysis – A Triple ...,[]
1,Executive Summary This technical analysis prov...,[]
2,For more information on CrowdStrike’s proactiv...,[]
3,NotPetya combines ransomware with the ability ...,[]
4,It spreads to Microsoft Windows machines using...,[T1210]
...,...,...
495,"The following is a list of services, hardcoded...",[]
496,QuickBooks.,[]
497,FCSmemtasmepocsPDVFSServiceQBCFMonitorServiceQ...,[]
498,#recycle$Recycle.,[]


In [5]:
import transformers
import torch

cuda = torch.device('cuda')

tokenizer = transformers.BertTokenizer.from_pretrained("allenai/scibert_scivocab_uncased", max_length=512)
bert = transformers.BertForSequenceClassification.from_pretrained('scibert_multi_label_model').to(cuda).train()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, shuffle=True)

def _load_data(x, y, batch_size=10):
    x_len, y_len = x.shape[0], y.shape[0]
    assert x_len == y_len
    for i in range(0, x_len, batch_size):
        slc = slice(i, i + batch_size)
        yield x[slc].to(cuda), y[slc].to(cuda)

def _tokenize(instances: list[str]):
    return tokenizer(instances, return_tensors='pt', padding='max_length', truncation=True, max_length=512).input_ids

def _encode_labels(labels):
    """:labels: should be the `labels` column (a Series) of the DataFrame"""
    return torch.Tensor(mlb.transform(labels.to_numpy()))

In [7]:
x_train = _tokenize(train['sentence'].tolist())

In [8]:
y_train = _encode_labels(train['labels'])
y_train

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

This cell contains the training loop. You may change the `NUM_EPOCHS` value to any integer you would like.

In [9]:
NUM_EPOCHS = 3

from statistics import mean

from tqdm import tqdm
from torch.optim import AdamW

optim = AdamW(bert.parameters(), lr=2e-5, eps=1e-8)

for epoch in range(NUM_EPOCHS):
    epoch_losses = []
    for x, y in tqdm(_load_data(x_train, y_train, batch_size=10)):
        bert.zero_grad()
        out = bert(x, attention_mask=x.ne(tokenizer.pad_token_id).to(int), labels=y)
        epoch_losses.append(out.loss.item())
        out.loss.backward()
        optim.step()
    print(f"epoch {epoch + 1} loss: {mean(epoch_losses)}")

40it [00:32,  1.22it/s]


epoch 1 loss: 0.011388333956710994


40it [00:33,  1.19it/s]


epoch 2 loss: 0.008742943072866183


40it [00:35,  1.13it/s]

epoch 3 loss: 0.006952952652500244


If the loss from the last iteration was not to your liking, do not re-run the previous cell. Uncomment the following cell and run it for however many additional epochs you would like.

In [ ]:
# NUM_EXTRA_EPOCHS = 1
# for epoch in range(NUM_EXTRA_EPOCHS):
#     epoch_losses = []
#     for x, y in tqdm(_load_data(x_train, y_train, batch_size=10)):
#         bert.zero_grad()
#         out = bert(x, attention_mask=x.ne(tokenizer.pad_token_id).to(int), labels=y)
#         epoch_losses.append(out.loss.item())
#         out.loss.backward()
#         optim.step()
#     print(f"epoch {epoch + 1} loss: {mean(epoch_losses)}")

The next cells evaluate the performance after the additional fine-tuning. The performance scores on the example data will be high, as the model has already been trained on most of these instances.

In [10]:
bert.eval()

x_test = _tokenize(test['sentence'].tolist())
y_test = _encode_labels(test['labels'])

batch_size = 20
preds = []

with torch.no_grad():
    for i in range(0, x_test.shape[0], batch_size):
        x = x_test[i : i + batch_size].to(cuda)
        out = bert(x, attention_mask=x.ne(tokenizer.pad_token_id).to(int))
        preds.extend(out.logits.to('cpu'))

binary_preds = torch.vstack(preds).sigmoid().gt(.5).to(int)

preds_series = pd.Series(mlb.inverse_transform(binary_preds)).apply(frozenset)
actual_series = pd.Series(mlb.inverse_transform(y_test)).apply(frozenset)
results = pd.concat({'predicted': preds_series, 'actual': actual_series}, axis=1)

results

,predicted,actual
0,(),()
1,(),()
2,(),()
3,(T1070.004),(T1070.004)
4,(),()
...,...,...
95,(),()
96,(),()
97,(),()
98,(T1574.002),(T1574.002)


In [11]:
tp = results.apply((lambda r: r.predicted & r.actual), axis=1).explode().value_counts()
fp = results.apply((lambda r: r.predicted - r.actual), axis=1).explode().value_counts()
fn = results.apply((lambda r: r.actual - r.predicted), axis=1).explode().value_counts()
counts = pd.concat({'tp': tp, 'fp': fp, 'fn': fn}, axis=1).fillna(0).astype(int)

support = actual_series.explode().value_counts().rename('#')

p = counts.tp.div(counts.tp + counts.fp).fillna(0)
r = counts.tp.div(counts.tp + counts.fn).fillna(0)
f1 = (2 * p * r) / (p + r)

scores = pd.concat({'P': p, 'R': r, 'F1': f1}, axis=1).fillna(0).sort_values(by='F1', ascending=False)

# calculate macro scores
scores.loc['(macro)'] = scores.mean()

# calculate micro scores
micro = counts.sum()
scores.loc['(micro)', 'P'] = mP = micro.tp / (micro.tp + micro.fp)
scores.loc['(micro)', 'R'] = mR = micro.tp / (micro.tp + micro.fn)
scores.loc['(micro)', 'F1'] = (2 * mP * mR) / (mP + mR)

scores.join(support)

,P,R,F1,#
T1106,1.000000,1.000000,1.000000,4.0
T1070.004,1.000000,1.000000,1.000000,2.0
T1078,1.000000,1.000000,1.000000,1.0
T1140,0.800000,1.000000,0.888889,4.0
T1055,0.666667,1.000000,0.800000,2.0
T1059.003,1.000000,0.666667,0.800000,3.0
T1027,0.800000,0.666667,0.727273,6.0
T1570,1.000000,0.500000,0.666667,2.0
T1218.011,1.000000,0.500000,0.666667,2.0
T1574.002,1.000000,0.500000,0.666667,2.0
